In [2]:
import seaborn           as sns
import numpy             as np
import pandas            as pd
import matplotlib.pyplot as plt
import tensorflow        as tf
from   tensorflow                import keras
from   tensorflow.keras          import layers
from   sklearn.model_selection   import train_test_split
from   sklearn.feature_selection import SelectKBest
from   sklearn.feature_selection import chi2
from   keras.models              import Sequential
from   keras.layers              import Dense, Dropout
from   keras.optimizers          import SGD
from   keras.callbacks           import EarlyStopping

Using TensorFlow backend.


In [3]:
# Upload dataset
from google.colab import files
uploaded = files.upload()

Saving hour.csv to hour.csv


In [0]:
# Load hours dataset
path = "./hour.csv"
dataset = pd.read_csv(path)

In [0]:
def generate_dummies(df, dummy_column):
    dummies = pd.get_dummies(df[dummy_column], prefix=dummy_column)
    df = pd.concat([df, dummies], axis=1)
    return df

X = pd.DataFrame.copy(dataset)
dummy_columns = ["season",     # season (1:springer, 2:summer, 3:fall, 4:winter)
                 "yr",          # year (0: 2011, 1:2012)
                 "mnth",        # month ( 1 to 12)
                 "hr",          # hour (0 to 23)
                 "weekday",     # weekday : day of the week
                 "weathersit"   # weathersit : 
                                 # - 1: Clear, Few clouds, Partly cloudy, Partly cloudy
                                 # - 2: Mist + Cloudy, Mist + Broken clouds, Mist + Few clouds, Mist
                                 # - 3: Light Snow, Light Rain + Thunderstorm + Scattered clouds, Light Rain + Scattered clouds
                                 # - 4: Heavy Rain + Ice Pallets + Thunderstorm + Mist, Snow + Fog
                ]
for dummy_column in dummy_columns:
    X = generate_dummies(X, dummy_column)

In [0]:
for dummy_column in dummy_columns:
  del X[dummy_column]

In [0]:
del X["instant"]
del X["dteday"]

y = X["cnt"]

del X["cnt"]
del X["registered"]
del X["casual"]

In [9]:
X.head()

,holiday,workingday,temp,atemp,hum,windspeed,season_1,season_2,season_3,season_4,yr_0,yr_1,mnth_1,mnth_2,mnth_3,mnth_4,mnth_5,mnth_6,mnth_7,mnth_8,mnth_9,mnth_10,mnth_11,mnth_12,hr_0,hr_1,hr_2,hr_3,hr_4,hr_5,hr_6,hr_7,hr_8,hr_9,hr_10,hr_11,hr_12,hr_13,hr_14,hr_15,hr_16,hr_17,hr_18,hr_19,hr_20,hr_21,hr_22,hr_23,weekday_0,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weathersit_1,weathersit_2,weathersit_3,weathersit_4
0,0,0,0.24,0.2879,0.81,0.0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0
1,0,0,0.22,0.2727,0.80,0.0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0
2,0,0,0.22,0.2727,0.80,0.0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0
3,0,0,0.24,0.2879,0.75,0.0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0
4,0,0,0.24,0.2879,0.75,0.0,1,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0


In [13]:
all_days = len(X) // 24
print("Total observations", len(X))
print("Total number of days", all_days)
days_for_training = int(all_days * 0.7)

X_train = X[0:days_for_training]
X_test = X[days_for_training:]

y_train = y[0:days_for_training]
y_test= y[days_for_training:]

Total observations 17379
Total number of days 724


In [11]:
print("Observations for training", len(X_train))
print("Observations for testing", len(X_test))
print("Some target values", y.head())

Observations for training 506
Observations for testing 16873
Some target values 0    16
1    40
2    32
3    13
4     1
Name: cnt, dtype: int64


In [14]:
print("Sum of X_train columns: ", len(X_train.columns.values))
print("Columns of X_train: \n", X_train.columns.values)

Sum of X_train columns:  59
Columns of X_train: 
 ['holiday' 'workingday' 'temp' 'atemp' 'hum' 'windspeed' 'season_1'
 'season_2' 'season_3' 'season_4' 'yr_0' 'yr_1' 'mnth_1' 'mnth_2' 'mnth_3'
 'mnth_4' 'mnth_5' 'mnth_6' 'mnth_7' 'mnth_8' 'mnth_9' 'mnth_10' 'mnth_11'
 'mnth_12' 'hr_0' 'hr_1' 'hr_2' 'hr_3' 'hr_4' 'hr_5' 'hr_6' 'hr_7' 'hr_8'
 'hr_9' 'hr_10' 'hr_11' 'hr_12' 'hr_13' 'hr_14' 'hr_15' 'hr_16' 'hr_17'
 'hr_18' 'hr_19' 'hr_20' 'hr_21' 'hr_22' 'hr_23' 'weekday_0' 'weekday_1'
 'weekday_2' 'weekday_3' 'weekday_4' 'weekday_5' 'weekday_6'
 'weathersit_1' 'weathersit_2' 'weathersit_3' 'weathersit_4']


In [0]:
#create model
model = Sequential()

#get number of columns in training data
n_cols = X.shape[1]

#add model layers
model.add(Dense(61, activation='relu', input_shape=(n_cols,)))
model.add(Dense(1, activation='linear'))

In [0]:
#compile model using mse as a measure of model performance
model.compile(optimizer='adam', 
              loss='mean_squared_error')

In [32]:
#set early stopping monitor so the model stops training when it won't improve anymore
early_stopping_monitor = EarlyStopping(patience=20)
#train model
model.fit(X_train,
          y_train, 
          validation_data = (X_test, y_test),
          epochs=150,
          callbacks=[early_stopping_monitor],
          verbose = 1)

Train on 506 samples, validate on 16873 samples
Epoch 1/150
506/506 [==============================] - 0s 512us/step - loss: 777.3364 - val_loss: 46960.5358
Epoch 2/150
506/506 [==============================] - 0s 521us/step - loss: 774.0998 - val_loss: 46924.7844
Epoch 3/150
506/506 [==============================] - 0s 519us/step - loss: 769.4143 - val_loss: 46917.3543
Epoch 4/150
506/506 [==============================] - 0s 508us/step - loss: 765.8626 - val_loss: 46828.5839
Epoch 5/150
506/506 [==============================] - 0s 505us/step - loss: 763.7062 - val_loss: 46955.8644
Epoch 6/150
506/506 [==============================] - 0s 503us/step - loss: 758.4059 - val_loss: 46671.0254
Epoch 7/150
506/506 [==============================] - 0s 533us/step - loss: 755.2168 - val_loss: 46716.5241
Epoch 8/150
506/506 [==============================] - 0s 512us/step - loss: 750.5761 - val_loss: 46738.9652
Epoch 9/150
506/506 [==============================] - 0s 518us/step - loss: 747